In [1]:
import numpy as np
import xarray as xr
import pandas as pd

import h5py
import os,json
from pprint import pprint

In [2]:
#change working directory
%cd ../

/home/jovyan/icepyx


In [3]:
%load_ext autoreload
%autoreload 2

from icepyx import icesat2data as ipd

### Choose a region for subsetting as well. Use the same region as in the core demo.

In [4]:
region_a = ipd.Icesat2Data('ATL07',[-170, 70, -130, 80],['2019-02-22','2019-02-23'], \
                           start_time='00:00:00', end_time='23:59:59', version='2')

/home/jovyan/icepyx/icepyx/core/icesat2data.py:115: UserWarning: Please note: as of 2020-05-05, a major reorganization of the core icepyx.icesat2data code may result in errors produced by now depricated functions. Please see our documentation pages or example notebooks for updates.
  warnings.warn("Please note: as of 2020-05-05, a major reorganization of the core icepyx.icesat2data code may result in errors produced by now depricated functions. Please see our documentation pages or example notebooks for updates.")


In [ ]:
region_a.earthdata_login('liuzheng','liuzheng@apl.uw.edu')

In [5]:
region_a.earthdata_login('jessica.scheick','jessica.scheick@maine.edu')

Earthdata Login password:  ········


### Use `latitude` only to reduce data request volume

In [6]:
var_dict = region_a.order_vars.append(beam_list=['gt2r'],var_list=['latitude'])
pprint(region_a.order_vars.wanted)

{'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'],
 'data_end_utc': ['ancillary_data/data_end_utc'],
 'data_start_utc': ['ancillary_data/data_start_utc'],
 'end_delta_time': ['ancillary_data/end_delta_time'],
 'granule_end_utc': ['ancillary_data/granule_end_utc'],
 'granule_start_utc': ['ancillary_data/granule_start_utc'],
 'latitude': ['gt2r/sea_ice_segments/latitude'],
 'sc_orient': ['orbit_info/sc_orient'],
 'sc_orient_time': ['orbit_info/sc_orient_time'],
 'start_delta_time': ['ancillary_data/start_delta_time']}


In [7]:
region_a.subsetparams(Coverage=region_a.order_vars.wanted)

{'time': '2019-02-22T00:00:00,2019-02-23T23:59:59',
 'Coverage': '/orbit_info/sc_orient,/orbit_info/sc_orient_time,/ancillary_data/atlas_sdp_gps_epoch,/ancillary_data/data_start_utc,/ancillary_data/data_end_utc,/ancillary_data/granule_start_utc,/ancillary_data/granule_end_utc,/ancillary_data/start_delta_time,/ancillary_data/end_delta_time,/gt2r/sea_ice_segments/latitude',
 'bbox': '-170,70,-130,80'}

In [12]:
region_a.reqparams

{'page_size': 2,
 'page_num': 1,
 'request_mode': 'async',
 'email': 'jessica.scheick@maine.edu',
 'include_meta': 'Y'}

### Setup download (found a bugs here but not sure how to fix)
* Use small `page_size` to generate more orders
* Bug?: HAVE to set `page_num` to 1. 
 * For repeated calling to order_granules, `page_num` might be set to larger than 1 at the end. 
 * With `page_num` larger than 1, the the total number of available granules is wrong and the resulting `page_num` and `page_size` are modified to values that do not make sense. 
 * Eg., set `page_num` to 2. 
 * Any idea why is this happenning?
 
 Yes: 
 when order_granules() is called, if it did not already have all of the required parameters for a download (versus a granule search), it was simply re-generating the reqparams, thus overwriting any that had already been set...
 I think I've corrected this issue, as well as updated the code so that it will actually check that an acceptable key has been submitted.

In [8]:
region_a.reqparams['page_size'] = 2
region_a.order_granules()

Total number of data order requests is  5  for  10  granules.
Data request  1  of  5  is submitting to NSIDC
order ID:  5000000693988
Initial status of your order request at NSIDC is:  processing
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order is:  complete_with_errors
NSIDC provided these error messages:
['166321086:NoMatchingData - No data found that matched subset constraints. '
 'Exit code 3.',
 'PT45.903S',
 'ICESAT2']
Your order is: complete_with_errors
Data request  2  of  5  is submitting to NS

In [9]:
region_a.granules.orderIDs

['5000000693988',
 '5000000693989',
 '5000000693991',
 '5000000693992',
 '5000000693993']

### Check the content of the restart file to see if it matches the records in region_a

In [10]:
!less .order_restart

{"orderIDs": ["5000000693988", "5000000693989", "5000000693991", "5000000693992", "5000000693993"]}
(END) (END)

### Start download and interrupt the kernel after downloading one order or two

In [11]:
region_a.download_granules('./down')

Beginning download of zipped output...
Data request 5000000693988 of  5  order(s) is complete.
Beginning download of zipped output...
Data request 5000000693989 of  5  order(s) is complete.


KeyboardInterrupt: 

### Now, restart. 

In [11]:
region_a.download_granules('./down',restart=True)

Restarting download ... 
Beginning download of zipped output...
Data request 5000000693989 of  4  order(s) is complete.
Beginning download of zipped output...


KeyboardInterrupt: 